In [ ]:
elastic_rods_dir = '../elastic_rods/python/'
weaving_dir = './'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, write_distance_to_linkage_mesh, set_design_parameters_from_topology)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis, render_video, combine_three_stage_video)


In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
rod_length = 0.21162395234664752
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
# width *= 0.5
# thickness *= 0.5

io = InputOrganizer('simple_hemoglobin_5_1', thickness, width, weaving_dir)
#  [ ear 214, ear 255, noseZ 39]

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
import pickle 
import gzip

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions()
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
curved_linkage_view.show()

### Stress Concentration from Pure Geometry

In [ ]:
ribbons = order_segments_by_ribbons(curved_linkage)
seg_list = [pair[0] for pair in ribbons[2]]

In [ ]:
def render_stress(filename, rangeMin = None, rangeMax = None):
    import vis
    from matplotlib import cm

    stress = np.array(curved_linkage.maxBendingStresses())
#     stress = curved_linkage.twistingStresses()
#     for i in range(len(stress)):
#         if i not in seg_list:
#             stress[i] *= 0
    if rangeMin == None:
        rangeMin = np.stack(stress).min()
    if rangeMax == None:
        rangeMax = np.stack(stress).max()
    print(rangeMin, rangeMax)
    print(np.stack(stress).min(), np.stack(stress).max())
    sf = vis.fields.ScalarField(curved_linkage, stress, colormap=cm.plasma, vmin=rangeMin, vmax=rangeMax)
    heights = curved_linkage.visualizationGeometryHeightColors()
    colors = sf.colors()
    
    # colors[heights < heights.mean()] = np.ones_like(sf.colors()[heights < heights.mean()]) * 0.5
    curved_linkage_view.update(scalarField = colors)

    renderCam = np.load('hemoglobin_stress_renderCam.npy')
    def renderToFile(path):
        orender = curved_linkage_view.offscreenRenderer(width=2048, height=2048)
        orender.setCameraParams(renderCam)
        orender.render()
        orender.save(path)
    renderToFile(filename)

In [ ]:
def render_distance(filename):
    def renderToFile(view, renderCam, path):
        orender = view.offscreenRenderer(width=2048, height=2048)
        orender.setCameraParams(renderCam)
        orender.render()
        orender.save(path)
    distance_color = write_distance_to_linkage_mesh(curved_linkage, 0.06418233901454702, None, return_distance_field = True)
    curved_linkage_view.update(scalarField = distance_color[:, :3])
    renderCam = np.load('hemoglobin_stress_renderCam.npy')
    renderToFile(curved_linkage_view, renderCam, filename)

In [ ]:
set_design_parameters_from_topology(curved_linkage, io)

In [ ]:
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
#     pass
   curved_linkage_view.update()
OPTS.gradTol = 1e-10

In [ ]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = curved_linkage.restLenFixedVars())
curved_linkage_view.update()

In [ ]:
render_stress('hemoglobin_geometry_fixed_joint.png', rangeMin = 0.08400753876564857, rangeMax = 37.316534657527576)
render_distance('hemoglobin_geometry_fixed_joint_dist.png')

In [ ]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])
curved_linkage_view.update()

In [ ]:
render_stress('hemoglobin_geometry_free_joint.png', rangeMin = 0.08400753876564857, rangeMax = 37.316534657527576)
render_distance('hemoglobin_geometry_dist.png')

In [ ]:
curved_linkage = pickle.load(gzip.open('mega_monster_optimization_diagram_results/tenth_round/simple_hemoglobin_5_1/simple_hemoglobin_5_1_stage_3.pkl.gz', 'r'))
curved_linkage.attraction_weight = 1e-5
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
curved_linkage_view.show()

In [ ]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])
curved_linkage_view.update()

In [ ]:
render_stress('hemoglobin_optimized.png', rangeMin = 0.08400753876564857, rangeMax = 37.316534657527576)
render_distance('hemoglobin_optimized_dist.png')

In [ ]:
distance_to_surface = np.array(curved_linkage.get_squared_distance_to_target_surface((curved_linkage.visualizationGeometry()[0]).flatten()))
distance_to_surface = np.sqrt(distance_to_surface)

In [ ]:
# renderCam = curved_linkage_view.getCameraParams()


In [ ]:
# np.save('hemoglobin_stress_renderCam.npy', renderCam)

In [ ]:
curved_linkage_view.setCameraParams((tuple(renderCam[0]), tuple(renderCam[1]), tuple(renderCam[2])))

In [ ]:
renderCam = np.load('optimization_diagram_results/simple_hemoglobin_5_1/video/renderCam.npy')

In [ ]:
renderCam[0], renderCam[2]

In [ ]:
renderCam

In [ ]:
# render_video(io.MODEL_NAME, 'stage_1', max(io.RIBBON_CS), renderCam)
# render_video(io.MODEL_NAME, 'stage_2', max(io.RIBBON_CS), renderCam)
# render_video(io.MODEL_NAME, 'stage_3', max(io.RIBBON_CS), renderCam)
# combine_three_stage_video(io.MODEL_NAME)

In [ ]:
# iterateData1 = np.load('optimization_diagram_results/simple_hemoglobin_5_1/simple_hemoglobin_5_1_stage_1.npy', allow_pickle=True)
# iterateData2 = np.load('optimization_diagram_results/sphere_1/sphere_1_stage_1.npy', allow_pickle=True)
# iterateData3 = np.load('optimization_diagram_results/simple_hemoglobin_5_1/simple_hemoglobin_5_1_stage_3.npy', allow_pickle=True)

In [ ]:
# sum([data_point['iteration_time'] for data_point in iterateData1])

In [ ]:
# sum([data_point['iteration_time'] for data_point in iterateData2])

In [ ]:
# sum([data_point['iteration_time'] for data_point in iterateData3][1:])

In [ ]:
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
curved_linkage_view.update()

In [ ]:
structure_view = get_structure_analysis_view(curved_linkage)
structure_view.show()

In [ ]:
from structural_analysis import weavingCrossingForceMagnitudes
cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
print("Normal Activation Threshold: ", np.percentile(cfm[:, 0], 75))

### Figure 9

Vector Plot

In [ ]:
def get_color_scheme(colors):
    '''
    Color scheme for force vectors.
    '''
#         cmap = plt.cm.plasma
    cmap = plt.cm.PuRd
    return cmap(colors)
#     import proplot as plot

#     # Colormap from named color
#     # The trailing '_r' makes the colormap go dark-to-light instead of light-to-dark
#     cmap1 = plot.Colormap('violet red', name='pacific', fade=100, space='hsl')
#     # The color map has 256 colors.
#     colors = np.round(colors * 256)
#     return plot.to_rgb(cmap1(colors), space = 'hsl')

In [ ]:
get_color_scheme(256)

In [ ]:
plt.cm.PuRd

In [ ]:
list_of_pickle_name = ['optimization_diagram_results/simple_hemoglobin_5_1/pickle/2_32.pkl.gz', 'optimization_diagram_results/simple_hemoglobin_5_1/pickle/3_34.pkl.gz']
list_of_output_name = ['{}_stage_2'.format(io.MODEL_NAME), '{}_stage_3'.format(io.MODEL_NAME)]

In [ ]:
import force_vector_visualization_helper
importlib.reload(force_vector_visualization_helper)
from force_vector_visualization_helper import write_force_vector_visualization_file
write_force_vector_visualization_file(list_of_pickle_name, list_of_output_name)

In [ ]:
j = curved_linkage.joint(0)

Density Plot

In [ ]:
import seaborn as sns

In [ ]:
separationForce = cfm[:, 0]

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 4, 1)
plt.xlim((separationForce.min(), max([0, separationForce.max()])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Separation Forces')
plt.xlabel('Separation Force Mag.')
plt.ylabel('Number of Crossings')
plt.hist(separationForce, 200);

In [ ]:
curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/2_28.pkl.gz', 'r'))
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
from structural_analysis import weavingCrossingForceMagnitudes
stage_2_cfm = weavingCrossingForceMagnitudes(curved_linkage, True)


curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/3_34.pkl.gz', 'r'))
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
from structural_analysis import weavingCrossingForceMagnitudes
stage_3_cfm = weavingCrossingForceMagnitudes(curved_linkage, True)

In [ ]:
stage_2_separation = stage_2_cfm[:, 0]
stage_3_separation = stage_3_cfm[:, 0]

In [ ]:
min_separation = min(min(stage_2_separation), min(stage_3_separation))
max_separation = max(max(stage_2_separation), max(stage_3_separation))

In [ ]:
import pandas as pd

In [ ]:
fig, host = plt.subplots()
data = np.transpose([stage_2_separation, stage_3_separation])
data = pd.DataFrame(data, columns=['Stage 2', 'Stage 3'])
sns.kdeplot(data = data, shade = True, cut = True)
plt.xlim((min_separation, max([0, max_separation])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Density of Separation Force Mag. at Crossings', fontsize = 14)
plt.xlabel('Separation Force Mag.', fontsize=14); 
plt.ylabel('Density', fontsize=14);
fig.set_size_inches((10, 4))
plt.tight_layout()
host.get_yaxis().set_ticks([])
plt.savefig('./separation_force_mag.svg', format='svg', dpi = 200)

In [ ]:
stage_2_tangential = stage_2_cfm[:, 1]
stage_3_tangential = stage_3_cfm[:, 1]

In [ ]:
min_tangential = min(min(stage_2_tangential), min(stage_3_tangential))
max_tangential = max(max(stage_2_tangential), max(stage_3_tangential))

In [ ]:
import pandas as pd

In [ ]:
fig, host = plt.subplots()
data = np.transpose([stage_2_tangential, stage_3_tangential])
data = pd.DataFrame(data, columns=['Stage 2', 'Stage 3'])
sns.kdeplot(data = data, shade = True, cut = True)
plt.xlim((min_tangential, max([0, max_tangential])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Density of Tangential Force Mag. at Crossings', fontsize = 14)
plt.xlabel('Tangential Force Mag.', fontsize=14); 
plt.ylabel('Density', fontsize=14);
fig.set_size_inches((10, 4))
plt.tight_layout()
host.get_yaxis().set_ticks([])
plt.savefig('./tangential_force_mag.svg', format='svg', dpi = 200)

In [ ]:
# save_path = os.getcwd()
# write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME + '_stage_2', io.RIBBON_NAME + '_stage_2', False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
# os.chdir(save_path)

### Videos

#### Generate images by running `python hemoglobin_vis_helper.py`

In [ ]:
import ffmpeg

#### Objective video for each stage

In [ ]:
framerate = 5
for stage_index in [1, 2, 3]:
    image_folder = '/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/video_images'
    video_name = os.path.join(image_folder, 'stage_{}_objective_stack.mp4'.format(str(stage_index)))
    if os.path.isfile(video_name):
        os.remove(video_name)
    (ffmpeg
        .input(os.path.join(image_folder, 'simple_hemoglobin_5_1_stage_{}_objective_stack_*.png'.format(str(stage_index))), pattern_type='glob', framerate=framerate)
        .output(video_name, crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
        .run()
    )

#### Objective combine videos

In [ ]:
import ffmpeg
image_folder = '/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/video_images'
file_list_path = os.path.join(image_folder, 'objective_stack_video_file_list.txt')
in1 = '{}_objective_stack.mp4'.format('stage_1')
in2 = '{}_objective_stack.mp4'.format('stage_2')
in3 = '{}_objective_stack.mp4'.format('stage_3')
with open(file_list_path, 'w') as f:
    f.write('file {}\nfile {}\nfile {}'.format(in1, in2, in3))
video_name = os.path.join(image_folder, 'combined_objective_stack.mp4')
if os.path.isfile(video_name):
    os.remove(video_name)
(ffmpeg
    .input(file_list_path, format='concat', safe=0)
    .output(video_name)
    .run()
)

#### Description video for each stage

In [ ]:
framerate = 5
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
for stage_index in [1, 2, 3]:
    image_folder = '/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/video_images'
    video_name = os.path.join(image_folder, 'stage_{}_components_description.mp4'.format(str(stage_index)))
    if os.path.isfile(video_name):
        os.remove(video_name)
    (ffmpeg
        .input(os.path.join(image_folder, 'simple_hemoglobin_5_1_stage_{}_components_description_*.png'.format(str(stage_index))), pattern_type='glob', framerate=framerate)
        .filter_("pad", **PAD_OPTIONS)
        .output(video_name, crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
        .run()
    )

#### Description combined video

In [ ]:
import ffmpeg
image_folder = '/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/video_images'
file_list_path = os.path.join(image_folder, 'components_description_video_file_list.txt')
in1 = '{}_components_description.mp4'.format('stage_1')
in2 = '{}_components_description.mp4'.format('stage_2')
in3 = '{}_components_description.mp4'.format('stage_3')
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
with open(file_list_path, 'w') as f:
    f.write('file {}\nfile {}\nfile {}'.format(in1, in2, in3))
video_name = os.path.join(image_folder, 'combined_components_description.mp4')
if os.path.isfile(video_name):
    os.remove(video_name)
(ffmpeg
    .input(file_list_path, format='concat', safe=0)
    .filter_("pad", **PAD_OPTIONS)
    .output(video_name)
    .run()
)

### Generate Curve Visualization

In [ ]:
limits = np.load('hemoglobin_ribbon_2_limits.npy')

#### Ribbon images for each stage

In [ ]:
for stage_index in [1, 2, 3]:
    data = np.load('/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/simple_hemoglobin_5_1_stage_{}.npy'.format(stage_index), allow_pickle = True)
    for i in range(len(data)):
        curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/{}_{}.pkl.gz'.format(stage_index, i), 'r'))
        get_all_curve_pattern(curved_linkage, thickness, io.SUBDIVISION_RESOLUTION, 'hemoglobin_opt_ribbon', 'png', target_ribbon_width = 5, flip_angles = False, select_ribbon_index = [2], iteration_index='{}_{:03d}'.format(stage_index, i), limits=limits)

In [ ]:
all_limits[0]

In [ ]:
# OPTS.niter = 1000
# fig, host = plt.subplots()
# host.set_aspect('equal')
# all_limits = []
# for stage_index in [1, 2, 3]:
#     data = np.load('/Users/yren/Develop/EPFL_LGG/weaving/weaving/optimization_diagram_results/simple_hemoglobin_5_1/simple_hemoglobin_5_1_stage_{}.npy'.format(stage_index), allow_pickle = True)
#     for i, curr_pt in enumerate(data):
#         curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/{}_{}.pkl.gz'.format(stage_index, i), 'r'))
#         top_x, top_y, bottom_x, bottom_y, joint_x, joint_y, limits = get_all_curve_pattern(curved_linkage, thickness, io.SUBDIVISION_RESOLUTION, 'hemoglobin_opt_ribbon', 'png', target_ribbon_width = 5, flip_angles = False, select_ribbon_index = [2], iteration_index='{:03d}'.format(i), return_png_data=True)
#         all_limits.append(limits)
#     #     plt.plot(top_x, top_y, '-', label='shifted up', linewidth = 0.2, color = 'black')
#     #     plt.plot(bottom_x, bottom_y, '-', label='shifted down', linewidth = 0.2, color = 'black')
#     #     plt.scatter(joint_x, joint_y, s = 0.1, facecolors='black', edgecolors='black')
#     # fig.savefig('all_ribbon.png', format='png', bbox_inches='tight', dpi = 200)

In [ ]:
# all_limits = np.array(all_limits)

In [ ]:
# limits = [np.max(all_limits[:, 0]), np.min(all_limits[:, 1]), np.max(all_limits[:, 2]), np.min(all_limits[:, 3])]

In [ ]:
# np.save('hemoglobin_ribbon_2_limits.npy', limits)

In [ ]:
np.max(all_limits, axis = 0)

In [ ]:
import ffmpeg

#### Ribbon video for each stage

In [ ]:
framerate = 5
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
for stage_index in [1, 2, 3]:
    image_folder = 'hemoglobin_opt_ribbon/'
    video_name = os.path.join(image_folder, 'stage_{}_ribbons.mp4'.format(str(stage_index)))
    if os.path.isfile(video_name):
        os.remove(video_name)
    (ffmpeg
        .input(os.path.join(image_folder, '2_{}_*.png'.format(str(stage_index))), pattern_type='glob', framerate=framerate)
        .filter_("pad", **PAD_OPTIONS)
        .output(video_name, crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
        .run()
    )

#### Ribbon combined video

In [ ]:
import ffmpeg
image_folder = 'hemoglobin_opt_ribbon/'
file_list_path = os.path.join(image_folder, 'ribbon_video_file_list.txt')
in1 = '{}_ribbons.mp4'.format('stage_1')
in2 = '{}_ribbons.mp4'.format('stage_2')
in3 = '{}_ribbons.mp4'.format('stage_3')
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
with open(file_list_path, 'w') as f:
    f.write('file {}\nfile {}\nfile {}'.format(in1, in2, in3))
video_name = os.path.join(image_folder, 'combined_ribbons.mp4')
if os.path.isfile(video_name):
    os.remove(video_name)
(ffmpeg
    .input(file_list_path, format='concat', safe=0)
    .filter_("pad", **PAD_OPTIONS)
    .output(video_name)
    .run()
)

### Combine Images in One Frame

In [ ]:
import cv2

In [ ]:
opt_image_root = 'optimization_diagram_results/simple_hemoglobin_5_1/video_images/'
stage_1_image_root = 'optimization_diagram_results/simple_hemoglobin_5_1/video/stage_1_images/'
stage_2_image_root = 'optimization_diagram_results/simple_hemoglobin_5_1/video/stage_2_images/'
stage_3_image_root = 'optimization_diagram_results/simple_hemoglobin_5_1/video/stage_3_images/'
ribbon_root = 'hemoglobin_opt_ribbon/'
output_image_root = 'optimization_diagram_results/simple_hemoglobin_5_1/panel_images/'
stage_1_num = 123
stage_2_num = 33
stage_3_num = 35

In [ ]:
123 + 33 + 35

In [ ]:
counter = 0
for i in range(stage_1_num):
    output_filename = os.path.join(output_image_root, 'stage_1/{:03d}.png'.format(i))
    counter += 1
    if os.path.isfile(output_filename):
        continue
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_2D_{:03d}.png'.format(i)))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_2D_-01.png'))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_2D_-01.png'))
    bottom_row = np.concatenate((img1, img2, img3), axis=1)
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_3D_{:03d}.png'.format(i)))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_3D_-01.png'))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_3D_-01.png'))
    top_row = np.concatenate((img1, img2, img3), axis=1)
    vis1 = np.concatenate((top_row, bottom_row))
    img1 = cv2.imread(os.path.join(stage_1_image_root, 'iter_{:03d}.png'.format(i)))
    img1[np.where(np.sum(img1, axis = 2) == 0)] = [255, 255, 255] 
    img2 = cv2.imread(os.path.join(ribbon_root, '2_1_{:03d}.png'.format(i)))
    img1 = cv2.resize(img1, (1515, 1515))
    img2 = cv2.resize(img2, (5037, 1515))
    vis2 = np.concatenate((img1, img2), axis=1)
    vis = np.concatenate((vis2, vis1))
    cv2.imwrite(output_filename, vis)
for i in range(stage_2_num):
    output_filename = os.path.join(output_image_root, 'stage_2/{:03d}.png'.format(i))
    counter += 1
    if os.path.isfile(output_filename):
        continue
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_2D_-01.png'))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_2D_{:03d}.png'.format(i)))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_2D_-01.png'))
    bottom_row = np.concatenate((img1, img2, img3), axis=1)
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_3D_-01.png'))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_3D_{:03d}.png'.format(i)))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_3D_-01.png'))
    top_row = np.concatenate((img1, img2, img3), axis=1)
    vis1 = np.concatenate((top_row, bottom_row))
    img1 = cv2.imread(os.path.join(stage_2_image_root, 'iter_{:03d}.png'.format(i)))
    img1[np.where(np.sum(img1, axis = 2) == 0)] = [255, 255, 255] 
    img2 = cv2.imread(os.path.join(ribbon_root, '2_2_{:03d}.png'.format(i)))
    img1 = cv2.resize(img1, (1515, 1515))
    img2 = cv2.resize(img2, (5037, 1515))
    vis2 = np.concatenate((img1, img2), axis=1)
    vis = np.concatenate((vis2, vis1))
    cv2.imwrite(output_filename, vis)
for i in range(stage_3_num):
    output_filename = os.path.join(output_image_root, 'stage_3/{:03d}.png'.format(i))
    counter += 1
    if os.path.isfile(output_filename):
        continue
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_2D_-01.png'))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_2D_-01.png'))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_2D_{:03d}.png'.format(i)))
    bottom_row = np.concatenate((img1, img2, img3), axis=1)
    img1 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_1_3D_-01.png'))
    img2 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_2_3D_-01.png'))
    img3 = cv2.imread(os.path.join(opt_image_root, 'simple_hemoglobin_5_1_stage_3_3D_{:03d}.png'.format(i)))
    top_row = np.concatenate((img1, img2, img3), axis=1)
    vis1 = np.concatenate((top_row, bottom_row))
    img1 = cv2.imread(os.path.join(stage_3_image_root, 'iter_{:03d}.png'.format(i)))
    img1[np.where(np.sum(img1, axis = 2) == 0)] = [255, 255, 255] 
    img2 = cv2.imread(os.path.join(ribbon_root, '2_3_{:03d}.png'.format(i)))
    img1 = cv2.resize(img1, (1515, 1515))
    img2 = cv2.resize(img2, (5037, 1515))
    vis2 = np.concatenate((img1, img2), axis=1)
    vis = np.concatenate((vis2, vis1))
    cv2.imwrite(output_filename, vis)

#### Panel Video

In [ ]:
import ffmpeg
framerate = 5
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
image_folder = output_image_root
video_name = os.path.join(image_folder, 'panel_video.mp4')
if os.path.isfile(video_name):
    os.remove(video_name)
(ffmpeg
    .input(os.path.join(image_folder, '*.png'), pattern_type='glob', framerate=framerate)
    .filter_("pad", **PAD_OPTIONS)
    .output(video_name, crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
    .run()
)

### Panel Video for each Stage

In [ ]:
framerates = [20, 5, 5]
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
for stage_index in [1, 2, 3]:
    image_folder = output_image_root
    video_name = os.path.join(image_folder, 'stage_{}.mp4'.format(str(stage_index)))
    if os.path.isfile(video_name):
        os.remove(video_name)
    (ffmpeg
        .input(os.path.join(image_folder, 'stage_{}/*.png'.format(str(stage_index))), pattern_type='glob', framerate=framerates[stage_index - 1])
        .filter_("pad", **PAD_OPTIONS)
        .output(video_name, crf=20, preset='slower', movflags='faststart', pix_fmt='yuv420p')
        .run()
    )

In [ ]:
import ffmpeg
image_folder = output_image_root
file_list_path = os.path.join(image_folder, 'video_file_list.txt')
in1 = '{}.mp4'.format('stage_1')
in2 = '{}.mp4'.format('stage_2')
in3 = '{}.mp4'.format('stage_3')
PAD_OPTIONS = {
    'width':'ceil(iw/2)*2',
    'height':'ceil(ih/2)*2',
    'x':'0',
    'y':'0',
    'color': 'white'
}
with open(file_list_path, 'w') as f:
    f.write('file {}\nfile {}\nfile {}'.format(in1, in2, in3))
video_name = os.path.join(image_folder, 'combined_panel.mp4')
if os.path.isfile(video_name):
    os.remove(video_name)
(ffmpeg
    .input(file_list_path, format='concat', safe=0)
    .filter_("pad", **PAD_OPTIONS)
    .output(video_name)
    .run()
)